In [11]:
import os
from dotenv import load_dotenv
load_dotenv()

from datetime import datetime
import random

from typing import TypedDict, Annotated, List, Literal, Dict
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.agents import create_agent
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langchain_tavily import TavilySearch
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode, tools_condition

from utils.utils import remove_think_content_for_qwen

In [2]:
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
llm = ChatGroq(model="qwen/qwen3-32b")

### Define state

In [3]:
class OrchestratorState(MessagesState):
    """State for the multi-agent system"""
    next_agent: str = ""
    data_retrieved: str = ""
    eligibility_check: bool = False
    content: str = ""
    communication: str = ""
    task_complete: bool = False
    current_task: str = ""

### Define Orchestrator Agent

In [9]:
def create_orchestrator_chain():
    """Create the orchestrator decision chain"""

    orchestrator_prompt = ChatPromptTemplate.from_messages([
        ("system", 
        """
        You are an orchestrating command managing a team of agents:
        1. Data Retriever: Fetches patient data from the database
        2. Eligibility Checker: Checks if patient has to be requested for a google review
        3. Content Generator: Develops a message to be sent to the patient
        4. Communication: Sends the message/email to the patient

        Based on the current state and conversation, decide which agent should work next.
        If the task is complete, respond with 'DONE'.

        Current state:
        - Has data: {has_data}
        - Has eligibility: {has_eligibility}
        - Has content: {has_content}
        - Message sent: {has_message}

        Respond with ONLY the agent name (data/eligibility/content/communication) or 'DONE'.
        """)
    ])

    return orchestrator_prompt | llm

In [12]:
decision = create_orchestrator_chain().invoke({
        "has_data": False,
        "has_eligibility": False,
        "has_content": False,
        "has_message": False,
    })

decision_text = remove_think_content_for_qwen(decision.content.strip().lower())
print(decision_text)

data


In [ ]:
def orchestrator_agent(state: OrchestratorState) -> Dict:
    """Orchestrator decides next agent using Groq LLM"""
    
    messages = state["messages"]
    task = messages[-1].content if messages else "No task"
    
    # Check what's been completed
    has_research = bool(state.get("research_data", ""))
    has_analysis = bool(state.get("analysis", ""))
    has_report = bool(state.get("final_report", ""))
    
    # Get LLM decision
    chain = create_supervisor_chain()
    decision = chain.invoke({
        "task": task,
        "has_research": has_research,
        "has_analysis": has_analysis,
        "has_report": has_report
    })
    
    # Parse decision
    decision_text = decision.content.strip().lower()
    print(decision_text)
    
    # Determine next agent
    if "done" in decision_text or has_report:
        next_agent = "end"
        supervisor_msg = "✅ Supervisor: All tasks complete! Great work team."
    elif "researcher" in decision_text or not has_research:
        next_agent = "researcher"
        supervisor_msg = "📋 Supervisor: Let's start with research. Assigning to Researcher..."
    elif "analyst" in decision_text or (has_research and not has_analysis):
        next_agent = "analyst"
        supervisor_msg = "📋 Supervisor: Research done. Time for analysis. Assigning to Analyst..."
    elif "writer" in decision_text or (has_analysis and not has_report):
        next_agent = "writer"
        supervisor_msg = "📋 Supervisor: Analysis complete. Let's create the report. Assigning to Writer..."
    else:
        next_agent = "end"
        supervisor_msg = "✅ Supervisor: Task seems complete."
    
    return {
        "messages": [AIMessage(content=supervisor_msg)],
        "next_agent": next_agent,
        "current_task": task
    }